<a href="https://colab.research.google.com/github/shawn-maybush/3D-Kinematics-Analytics/blob/main/quaternions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Inverse Kinematics with Dual Quaternions



# Set Up

##System Check

In [1]:
from psutil import *
# This code will return the number of CPU
print("Number of CPU: ", cpu_count())
# This code will return the CPU info
import psutil
split_bar = '='*20
memory_info = psutil.virtual_memory()._asdict()
print(f"{split_bar} Memory Usage {split_bar}")
for k,v in memory_info.items():
  print(k, v)
print(f"{split_bar} CPU Usage {split_bar}")
print(f"CPU percent: {psutil.cpu_percent()}%")

Number of CPU:  8
==================== Memory Usage ====================
total 54754004992
available 53476298752
percent 2.3
used 686059520
free 49795239936
active 400351232
inactive 4196605952
buffers 182792192
cached 4089913344
shared 1531904
slab 194039808
==================== CPU Usage ====================
CPU percent: 12.1%


##Define Forward Kinematics

In [2]:
import sympy
from math import pi, sqrt, degrees, radians
import numpy as np
from IPython.display import display, Math
import altair as alt
import pandas as pd
import ipyparallel as ipp
from sympy.simplify.radsimp import collect_sqrt

In [3]:
cluster = ipp.Cluster(n=8)
rc = cluster.start_and_connect_sync()
dv = rc[:]
lv = rc.load_balanced_view()

Starting 8 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/8 [00:00<?, ?engine/s]

In [4]:
with rc[:].sync_imports():
  import sympy
  from math import pi, sqrt, degrees, radians
  import numpy
  from IPython.display import display, Math
  import altair
  import pandas
  from sympy.simplify.radsimp import collect_sqrt

importing sympy on engine(s)
importing pi,sqrt,degrees,radians from math on engine(s)
importing numpy on engine(s)
importing display,Math from IPython.display on engine(s)
importing altair on engine(s)
importing pandas on engine(s)
importing collect_sqrt from sympy.simplify.radsimp on engine(s)


In [5]:
def q_piecewise_simplify(q):
    q_scalar = q.scalar_part()
    q_vector = q.to_Matrix(vector_only=True).T.tolist()
    all = [q_scalar] + q_vector[0]
    all = lv.map_sync(lambda x: sympy.simplify(x), all)
    result = sympy.Quaternion(*all)
    return result

def q_piecewise_expand(q):
    q_scalar = q.scalar_part()
    q_vector = q.to_Matrix(vector_only=True).T.tolist()
    all = [q_scalar] + q_vector[0]
    all = lv.map_sync(lambda x: sympy.expand(x), all)
    result = sympy.Quaternion(*all)
    return result

def q_piecewise_trigsimp(q):
    q_scalar = q.scalar_part()
    q_vector = q.to_Matrix(vector_only=True).T.tolist()
    all = [q_scalar] + q_vector[0]
    all = lv.map_sync(lambda x: sympy.trigsimp(x), all)
    result = sympy.Quaternion(*all)
    return result

def q_piecewise_collect_sqrt(q):
    q_scalar = q.scalar_part()
    q_vector = q.to_Matrix(vector_only=True).T.tolist()
    all = [q_scalar] + q_vector[0]
    all = lv.map_sync(lambda x: collect_sqrt(x), all)
    result = sympy.Quaternion(*all)
    return result

def q_piecewise_cancel(q):
    q_scalar = q.scalar_part()
    q_vector = q.to_Matrix(vector_only=True).T.tolist()
    all = [q_scalar] + q_vector[0]
    all = lv.map_sync(lambda x: sympy.cancel(x), all)
    result = sympy.Quaternion(*all)
    return result

def q_piecewise_expand_log(q):
    q_scalar = q.scalar_part()
    q_vector = q.to_Matrix(vector_only=True).T.tolist()
    all = [q_scalar] + q_vector[0]
    all = lv.map_sync(lambda x: sympy.expand_log(x), all)
    result = sympy.Quaternion(*all)
    return result

In [6]:
def dualq_piecewise_simplify(dq):

  r = dq[0]
  d = dq[1]
  r_scalar = r.scalar_part()
  r_vector = r.to_Matrix(vector_only=True).T.tolist()

  r_list = [r_scalar] + r_vector[0]

  d_scalar = d.scalar_part()
  d_vector = d.to_Matrix(vector_only=True).T.tolist()
  d_list = [d_scalar] + d_vector[0]

  all = r_list + d_list

  all = lv.map_sync(lambda x: sympy.simplify(x), all)

  real_part = sympy.Quaternion(*all[:4])
  dual_part = sympy.Quaternion(*all[4:])

  result = sympy.Matrix([real_part, dual_part])

  return result

def dualq_piecewise_expand(dq):

  r = dq[0]
  d = dq[1]
  r_scalar = r.scalar_part()
  r_vector = r.to_Matrix(vector_only=True).T.tolist()

  r_list = [r_scalar] + r_vector[0]

  d_scalar = d.scalar_part()
  d_vector = d.to_Matrix(vector_only=True).T.tolist()
  d_list = [d_scalar] + d_vector[0]

  all = r_list + d_list

  all = lv.map_sync(lambda x: sympy.expand(x, basic=True), all)

  real_part = sympy.Quaternion(*all[:4])
  dual_part = sympy.Quaternion(*all[4:])

  result = sympy.Matrix([real_part, dual_part])

  return result

def dualq_piecewise_expand_trig(dq):

  r = dq[0]
  d = dq[1]
  r_scalar = r.scalar_part()
  r_vector = r.to_Matrix(vector_only=True).T.tolist()

  r_list = [r_scalar] + r_vector[0]

  d_scalar = d.scalar_part()
  d_vector = d.to_Matrix(vector_only=True).T.tolist()
  d_list = [d_scalar] + d_vector[0]

  all = r_list + d_list

  all = lv.map_sync(lambda x: sympy.expand_trig(x), all)

  real_part = sympy.Quaternion(*all[:4])
  dual_part = sympy.Quaternion(*all[4:])

  result = sympy.Matrix([real_part, dual_part])

  return result

def dualq_piecewise_factor(dq):

  r = dq[0]
  d = dq[1]
  r_scalar = r.scalar_part()
  r_vector = r.to_Matrix(vector_only=True).T.tolist()

  r_list = [r_scalar] + r_vector[0]

  d_scalar = d.scalar_part()
  d_vector = d.to_Matrix(vector_only=True).T.tolist()
  d_list = [d_scalar] + d_vector[0]

  all = r_list + d_list

  all = lv.map_sync(lambda x: sympy.factor(x, deep=False), all)

  real_part = sympy.Quaternion(*all[:4])
  dual_part = sympy.Quaternion(*all[4:])

  result = sympy.Matrix([real_part, dual_part])

  return result


def dualq_piecewise_trigsimp(dq):

  r = dq[0]
  d = dq[1]
  r_scalar = r.scalar_part()
  r_vector = r.to_Matrix(vector_only=True).T.tolist()

  r_list = [r_scalar] + r_vector[0]

  d_scalar = d.scalar_part()
  d_vector = d.to_Matrix(vector_only=True).T.tolist()
  d_list = [d_scalar] + d_vector[0]

  all = r_list + d_list


  all = lv.map_sync(lambda x: sympy.trigsimp(x), all)

  real_part = sympy.Quaternion(*all[:4])
  dual_part = sympy.Quaternion(*all[4:])

  result = sympy.Matrix([real_part, dual_part])

  return result

def dualq_piecewise_cancel(dq):

  r = dq[0]
  d = dq[1]
  r_scalar = r.scalar_part()
  r_vector = r.to_Matrix(vector_only=True).T.tolist()

  r_list = [r_scalar] + r_vector[0]

  d_scalar = d.scalar_part()
  d_vector = d.to_Matrix(vector_only=True).T.tolist()
  d_list = [d_scalar] + d_vector[0]

  all = r_list + d_list


  all = lv.map_sync(lambda x: sympy.cancel(x), all)

  real_part = sympy.Quaternion(*all[:4])
  dual_part = sympy.Quaternion(*all[4:])

  result = sympy.Matrix([real_part, dual_part])

  return result

def dualq_piecewise_collect_sqrt(dq):

  r = dq[0]
  d = dq[1]
  r_scalar = r.scalar_part()
  r_vector = r.to_Matrix(vector_only=True).T.tolist()

  r_list = [r_scalar] + r_vector[0]

  d_scalar = d.scalar_part()
  d_vector = d.to_Matrix(vector_only=True).T.tolist()
  d_list = [d_scalar] + d_vector[0]

  all = r_list + d_list


  all = lv.map_sync(lambda x: collect_sqrt(x), all)

  real_part = sympy.Quaternion(*all[:4])
  dual_part = sympy.Quaternion(*all[4:])

  result = sympy.Matrix([real_part, dual_part])

  return result

def dualq_piecewise_separatevars(dq):

  r = dq[0]
  d = dq[1]
  r_scalar = r.scalar_part()
  r_vector = r.to_Matrix(vector_only=True).T.tolist()

  r_list = [r_scalar] + r_vector[0]

  d_scalar = d.scalar_part()
  d_vector = d.to_Matrix(vector_only=True).T.tolist()
  d_list = [d_scalar] + d_vector[0]

  all = r_list + d_list


  all = lv.map_sync(lambda x: sympy.separatevars(x), all)

  real_part = sympy.Quaternion(*all[:4])
  dual_part = sympy.Quaternion(*all[4:])

  result = sympy.Matrix([real_part, dual_part])

  return result

def dualq_piecewise_collect(dq, c):

  r = dq[0]
  d = dq[1]
  r_scalar = r.scalar_part()
  r_vector = r.to_Matrix(vector_only=True).T.tolist()

  r_list = [r_scalar] + r_vector[0]

  d_scalar = d.scalar_part()
  d_vector = d.to_Matrix(vector_only=True).T.tolist()
  d_list = [d_scalar] + d_vector[0]

  all = r_list + d_list


  all = lv.map_sync(lambda x: sympy.collect(x, c), all)

  real_part = sympy.Quaternion(*all[:4])
  dual_part = sympy.Quaternion(*all[4:])

  result = sympy.Matrix([real_part, dual_part])

  return result

def dualq_piecewise_sqrtdenest(dq):

  r = dq[0]
  d = dq[1]
  r_scalar = r.scalar_part()
  r_vector = r.to_Matrix(vector_only=True).T.tolist()

  r_list = [r_scalar] + r_vector[0]

  d_scalar = d.scalar_part()
  d_vector = d.to_Matrix(vector_only=True).T.tolist()
  d_list = [d_scalar] + d_vector[0]

  all = r_list + d_list


  all = lv.map_sync(lambda x: sympy.sqrtdenest(x), all)

  real_part = sympy.Quaternion(*all[:4])
  dual_part = sympy.Quaternion(*all[4:])

  result = sympy.Matrix([real_part, dual_part])

  return result

In [7]:
def dualq_mul(*args):


    if len(args) < 2:
        raise Exception("Need at least 2 dual quaternions to multiply")

    result = args[0]
    for i in range(1, len(args)):

        dq_i = args[i]

        r1 = result[0]
        d1 = result[1]

        r2 = dq_i[0]
        d2 = dq_i[1]

        real_part = r1.mul(r2)
        dual_part = (r1.mul(d2)).add(d1.mul(r2))

        result = sympy.Matrix([real_part, dual_part])


    return result


def q_from_axis_angle(axis, angle):

  kappa_r = (sympy.series(sympy.sin(angle/2), angle, 0, 5).removeO()/angle)/sympy.sqrt(axis.dot(axis))

  return sympy.Quaternion(sympy.cos(angle/2), *(kappa_r * axis))



def q_dot_product(q1, q2):
    q1_scalar = q1.scalar_part()
    q1_vector = q1.to_Matrix(vector_only=True)
    q2_scalar = q2.scalar_part()
    q2_vector = q2.to_Matrix(vector_only=True)

    return q1_scalar * q2_scalar + q1_vector.dot(q2_vector)

def q_log(q):

    q = q.normalize()

    qw, qv = q.scalar_part(), q.to_Matrix(vector_only=True)

    qv_norm = qv.norm()

    v = (2*sympy.acos(qw) / qv_norm) * qv

    return sympy.Quaternion(0, *v)

def dualq_diff_alt(dq, wrt):

    r = dq[0]
    t = 2 * dq[1].mul(r.conjugate())

    r_dot = r.diff(wrt)

    real_part = r_dot
    dual_part = 0.5 * t.mul(r_dot)


    result = sympy.Matrix([real_part, dual_part])
    result = dualq_piecewise_expand(result)
    result = dualq_piecewise_simplify(result)

    return result


def dualq_diff(dq, wrt):

    r = dq[0]
    d = dq[1]

    r_dot = r.diff(wrt)

    d_dot = d.diff(wrt)

    real_part = r_dot
    dual_part_1 = d_dot.mul(r)
    dual_part_2 = r_dot.mul(d)
    dual_part = dual_part_1.add(dual_part_2)


    result = sympy.Matrix([real_part, dual_part])
    result = dualq_piecewise_expand(result)

    return result


def extract_linear_velocity(dq_diff):


    r_dot = dq_diff[0]
    d_dot = dq_diff[1]

    v_t_1 = (2 * d_dot)

    v = v_t_1.mul(r_dot.conjugate())
    v = v.to_Matrix(vector_only=True)

    return v

def q_inverse(q):
    q_scalar = q.scalar_part()
    q_vector = q.to_Matrix(vector_only=True)

    q_norm = sympy.sqrt(q_scalar**2 + q_vector.dot(q_vector))

    result = q.conjugate()/q_norm**2

    return result

def q_subtract(q1, q2):
    q3_scalar = q1.scalar_part() - q2.scalar_part()
    q3_vector = q1.to_Matrix(vector_only=True) - q2.to_Matrix(vector_only=True)

    result = sympy.Quaternion(q3_scalar, *q3_vector)

    return result


In [8]:
L1 = 30.0
L2 = 90.0
L3 = 110.0
theta_1, theta_2, theta_3 = sympy.symbols('theta_1 theta_2 theta_3', real=True)
ax_1, ax_2, ax_3 = sympy.symbols('ax_1 ax_2 ax_3', real=True)
ay_1, ay_2, ay_3 = sympy.symbols('ay_1 ay_2 ay_3', real=True)
az_1, az_2, az_3 = sympy.symbols('az_1 az_2 az_3', real=True)

In [9]:
r_c = sympy.Quaternion.from_axis_angle([ax_1, ay_1, az_1],theta_1)
t_c = sympy.Quaternion(0, 0, 0, 0)
d_c = sympy.Rational(0.5) * t_c.mul(r_c)
Q_c = sympy.Matrix([r_c, d_c])
Q_c = dualq_piecewise_simplify(Q_c)
Q_c

Matrix([
[cos(theta_1/2) + ax_1*sin(theta_1/2)/sqrt(ax_1**2 + ay_1**2 + az_1**2)*i + ay_1*sin(theta_1/2)/sqrt(ax_1**2 + ay_1**2 + az_1**2)*j + az_1*sin(theta_1/2)/sqrt(ax_1**2 + ay_1**2 + az_1**2)*k],
[                                                                                                                                                                         0 + 0*i + 0*j + 0*k]])

In [10]:
r_ci = q_from_axis_angle(sympy.Matrix([ax_1, ay_1, az_1]),theta_1)
t_ci = sympy.Quaternion(0, 0, 0, 0)
d_ci = sympy.Rational(0.5) * t_ci.mul(r_ci)
Q_ci = sympy.Matrix([r_ci, d_ci])
Q_ci = dualq_piecewise_simplify(Q_ci)
Q_ci

Matrix([
[cos(theta_1/2) + ax_1*(24 - theta_1**2)/(48*sqrt(ax_1**2 + ay_1**2 + az_1**2))*i + ay_1*(24 - theta_1**2)/(48*sqrt(ax_1**2 + ay_1**2 + az_1**2))*j + az_1*(24 - theta_1**2)/(48*sqrt(ax_1**2 + ay_1**2 + az_1**2))*k],
[                                                                                                                                                                                                 0 + 0*i + 0*j + 0*k]])

In [11]:
r_f = sympy.Quaternion.from_axis_angle([ax_2, ay_2, az_2],theta_2)
t_f = sympy.Quaternion(0, L1, 0, 0)
d_f = sympy.Rational(0.5) * t_f.mul(r_f)
Q_f = sympy.Matrix([r_f,d_f])
Q_f = dualq_piecewise_simplify(Q_f)
Q_f

Matrix([
[                          cos(theta_2/2) + ax_2*sin(theta_2/2)/sqrt(ax_2**2 + ay_2**2 + az_2**2)*i + ay_2*sin(theta_2/2)/sqrt(ax_2**2 + ay_2**2 + az_2**2)*j + az_2*sin(theta_2/2)/sqrt(ax_2**2 + ay_2**2 + az_2**2)*k],
[(-15.0*ax_2*sin(theta_2/2)/sqrt(ax_2**2 + ay_2**2 + az_2**2)) + 15.0*cos(theta_2/2)*i + (-15.0*az_2*sin(theta_2/2)/sqrt(ax_2**2 + ay_2**2 + az_2**2))*j + 15.0*ay_2*sin(theta_2/2)/sqrt(ax_2**2 + ay_2**2 + az_2**2)*k]])

In [12]:
r_fi = q_from_axis_angle(sympy.Matrix([ax_2, ay_2, az_2]),theta_2)
t_fi = sympy.Quaternion(0, L1, 0, 0)
d_fi = sympy.Rational(0.5) * t_fi.mul(r_fi)
Q_fi = sympy.Matrix([r_fi,d_fi])
Q_fi = dualq_piecewise_simplify(Q_fi)
Q_fi

Matrix([
[              cos(theta_2/2) + ax_2*(24 - theta_2**2)/(48*sqrt(ax_2**2 + ay_2**2 + az_2**2))*i + ay_2*(24 - theta_2**2)/(48*sqrt(ax_2**2 + ay_2**2 + az_2**2))*j + az_2*(24 - theta_2**2)/(48*sqrt(ax_2**2 + ay_2**2 + az_2**2))*k],
[0.3125*ax_2*(theta_2**2 - 24)/sqrt(ax_2**2 + ay_2**2 + az_2**2) + 15.0*cos(theta_2/2)*i + 0.3125*az_2*(theta_2**2 - 24)/sqrt(ax_2**2 + ay_2**2 + az_2**2)*j + (-0.3125*ay_2*(theta_2**2 - 24)/sqrt(ax_2**2 + ay_2**2 + az_2**2))*k]])

In [13]:
r_t = sympy.Quaternion.from_axis_angle([ax_3, ay_3, az_3],theta_3)
t_t = sympy.Quaternion(0, L2, 0, 0)
d_t = sympy.Rational(0.5) * t_t.mul(r_t)
Q_t = sympy.Matrix([r_t, d_t])
Q_t = dualq_piecewise_simplify(Q_t)
Q_t

Matrix([
[                          cos(theta_3/2) + ax_3*sin(theta_3/2)/sqrt(ax_3**2 + ay_3**2 + az_3**2)*i + ay_3*sin(theta_3/2)/sqrt(ax_3**2 + ay_3**2 + az_3**2)*j + az_3*sin(theta_3/2)/sqrt(ax_3**2 + ay_3**2 + az_3**2)*k],
[(-45.0*ax_3*sin(theta_3/2)/sqrt(ax_3**2 + ay_3**2 + az_3**2)) + 45.0*cos(theta_3/2)*i + (-45.0*az_3*sin(theta_3/2)/sqrt(ax_3**2 + ay_3**2 + az_3**2))*j + 45.0*ay_3*sin(theta_3/2)/sqrt(ax_3**2 + ay_3**2 + az_3**2)*k]])

In [14]:
r_ti = q_from_axis_angle(sympy.Matrix([ax_3, ay_3, az_3]),theta_3)
t_ti = sympy.Quaternion(0, L2, 0, 0)
d_ti = sympy.Rational(0.5) * t_ti.mul(r_ti)
Q_ti = sympy.Matrix([r_ti, d_ti])
Q_ti = dualq_piecewise_simplify(Q_ti)
Q_ti

Matrix([
[              cos(theta_3/2) + ax_3*(24 - theta_3**2)/(48*sqrt(ax_3**2 + ay_3**2 + az_3**2))*i + ay_3*(24 - theta_3**2)/(48*sqrt(ax_3**2 + ay_3**2 + az_3**2))*j + az_3*(24 - theta_3**2)/(48*sqrt(ax_3**2 + ay_3**2 + az_3**2))*k],
[0.9375*ax_3*(theta_3**2 - 24)/sqrt(ax_3**2 + ay_3**2 + az_3**2) + 45.0*cos(theta_3/2)*i + 0.9375*az_3*(theta_3**2 - 24)/sqrt(ax_3**2 + ay_3**2 + az_3**2)*j + (-0.9375*ay_3*(theta_3**2 - 24)/sqrt(ax_3**2 + ay_3**2 + az_3**2))*k]])

In [15]:
r_e = sympy.Quaternion(1, 0, 0, 0)
t_e = sympy.Quaternion(0.0, L3, 0, 0)
d_e = sympy.Rational(0.5) * t_e.mul(r_e)
Q_e = sympy.Matrix([r_e, d_e])
Q_e = dualq_piecewise_expand(Q_e)
Q_e

Matrix([
[   1 + 0*i + 0*j + 0*k],
[0 + 55.0*i + 0*j + 0*k]])

In [16]:
test_subs = {theta_1:0, ax_1:0, ay_1:0, az_1:1, theta_2:0, ax_2:0, ay_2:1, az_2:0, theta_3:0, ax_3:0, ay_3:1, az_3:0}

In [17]:
foot_pose_dqi = dualq_mul(Q_c, Q_f, Q_t, Q_e)
foot_pose_dqi = dualq_piecewise_expand(foot_pose_dqi)
foot_position_FK = (2 * foot_pose_dqi[1].mul(foot_pose_dqi[0].conjugate()))
foot_position_FK = foot_position_FK.to_Matrix(vector_only=True)
foot_orientation_FK = foot_pose_dqi[0].to_Matrix(vector_only=True)

In [18]:
foot_orientation_FK.subs(test_subs)

Matrix([
[0],
[0],
[0]])

In [19]:
q_ew = dualq_mul(Q_c, Q_f, Q_t, Q_e)
q_ew_r = q_ew[0]
q_ew_t = q_ew[1].mul(q_ew_r.conjugate())

In [20]:
o_w = q_log(q_ew[0])

In [21]:
q_ew_r_1, q_ew_r_2, q_ew_r_3 = sympy.symbols('q_ew_r_1 q_ew_r_2 q_ew_r_3')
q_ew_r_v = q_ew_r.to_Matrix(vector_only=True)
q_ew_r_1_value = q_ew_r_v[0]
q_ew_r_2_value = q_ew_r_v[1]
q_ew_r_3_value = q_ew_r_v[2]

subs_dict = {q_ew_r_1_value:q_ew_r_1, q_ew_r_2_value: q_ew_r_2, q_ew_r_3_value: q_ew_r_3}
q_ew_r_subbed = q_ew_r.subs(subs_dict)

o_l_dot_r_1 = q_ew_r_subbed.diff(q_ew_r_1)
o_l_dot_r_2 = q_ew_r_subbed.diff(q_ew_r_2)
o_l_dot_r_3 = q_ew_r_subbed.diff(q_ew_r_3)

In [22]:
o_l_dot_r_3.conjugate()

0 + 0*i + 0*j + (-1)*k

In [23]:
test_subs = {theta_1:sympy.pi/8, ax_1:0, ay_1:0, az_1:1, theta_2:0, ax_2:0, ay_2:1, az_2:0, theta_3:0, ax_3:0, ay_3:1, az_3:0}

##Joint 1

In [24]:
r_dot_theta_1 = dualq_diff_alt(Q_c, theta_1)
r_dot_ax_1 = dualq_diff_alt(Q_c, ax_1)
r_dot_ay_1 = dualq_diff_alt(Q_c, ay_1)
r_dot_az_1 = dualq_diff_alt(Q_c, az_1)

qw_e_dot_theta_1 = dualq_mul(r_dot_theta_1, Q_f, Q_t, Q_e)
qw_e_dot_ax_1 = dualq_mul(r_dot_ax_1, Q_f, Q_t, Q_e)
qw_e_dot_ay_1 = dualq_mul(r_dot_ay_1, Q_f, Q_t, Q_e)
qw_e_dot_az_1 = dualq_mul(r_dot_az_1, Q_f, Q_t, Q_e)

###Joint 1 Angular velocity (ω)

In [25]:
w_theta_1_1 = ((o_l_dot_r_1.conjugate()).mul(qw_e_dot_theta_1[0])).scalar_part()
w_theta_1_2 = ((o_l_dot_r_2.conjugate()).mul(qw_e_dot_theta_1[0])).scalar_part()
w_theta_1_3 = ((o_l_dot_r_3.conjugate()).mul(qw_e_dot_theta_1[0])).scalar_part()
w_theta_1 = sympy.Matrix([w_theta_1_1, w_theta_1_2, w_theta_1_3]) * 2

w_ax_1_1 = ((o_l_dot_r_1.conjugate()).mul(qw_e_dot_ax_1[0])).scalar_part()
w_ax_1_2 = ((o_l_dot_r_2.conjugate()).mul(qw_e_dot_ax_1[0])).scalar_part()
w_ax_1_3 = ((o_l_dot_r_3.conjugate()).mul(qw_e_dot_ax_1[0])).scalar_part()
w_ax_1 = sympy.Matrix([w_ax_1_1, w_ax_1_2, w_ax_1_3]) * 2

w_ay_1_1 = ((o_l_dot_r_1.conjugate()).mul(qw_e_dot_ay_1[0])).scalar_part()
w_ay_1_2 = ((o_l_dot_r_2.conjugate()).mul(qw_e_dot_ay_1[0])).scalar_part()
w_ay_1_3 = ((o_l_dot_r_3.conjugate()).mul(qw_e_dot_ay_1[0])).scalar_part()
w_ay_1 = sympy.Matrix([w_ay_1_1, w_ay_1_2, w_ay_1_3]) * 2

w_az_1_1 = ((o_l_dot_r_1.conjugate()).mul(qw_e_dot_az_1[0])).scalar_part()
w_az_1_2 = ((o_l_dot_r_2.conjugate()).mul(qw_e_dot_az_1[0])).scalar_part()
w_az_1_3 = ((o_l_dot_r_3.conjugate()).mul(qw_e_dot_az_1[0])).scalar_part()
w_az_1 = sympy.Matrix([w_az_1_1, w_az_1_2, w_az_1_3]) * 2


In [26]:
test = w_theta_1.subs(test_subs)
display(Math(sympy.latex(test)))

<IPython.core.display.Math object>

### Joint 1 Linear velocity (v)

In [ ]:
'''
v_theta_1_a = 2 * qw_e_dot_theta_1[1]
v_theta_1_b = q_ew_t.mul(qw_e_dot_theta_1[0])
v_theta_1 = (q_subtract(v_theta_1_a, v_theta_1_b)).mul(q_ew_r.conjugate())
v_theta_1 = v_theta_1.to_Matrix(vector_only=True)

v_ax_1_a = 2 * qw_e_dot_ax_1[1]
v_ax_1_b = q_ew_t.mul(qw_e_dot_ax_1[0])
v_ax_1 = (q_subtract(v_ax_1_a,v_ax_1_b)).mul(q_ew_r.conjugate())
v_ax_1 = v_ax_1.to_Matrix(vector_only=True)

v_ay_1_a = 2 * qw_e_dot_ay_1[1]
v_ay_1_b = q_ew_t.mul(qw_e_dot_ay_1[0])
v_ay_1 = (q_subtract(v_ay_1_a, v_ay_1_b)).mul(q_ew_r.conjugate())
v_ay_1 = v_ay_1.to_Matrix(vector_only=True)

v_az_1_a = 2 * qw_e_dot_az_1[1]
v_az_1_b = q_ew_t.mul(qw_e_dot_az_1[0])
v_az_1 = (q_subtract(v_az_1_a, v_az_1_b)).mul(q_ew_r.conjugate())
v_az_1 = v_az_1.to_Matrix(vector_only=True)
'''
v_theta_1 = (2 * qw_e_dot_theta_1[1]).mul(qw_e_dot_theta_1[0].conjugate())
v_theta_1 = v_theta_1.to_Matrix(vector_only=True)
v_ax_1 = (2 * qw_e_dot_ax_1[1]).mul(qw_e_dot_ax_1[0].conjugate())
v_ax_1 = v_ax_1.to_Matrix(vector_only=True)
v_ay_1 = (2 * qw_e_dot_ay_1[1]).mul(qw_e_dot_ay_1[0].conjugate())
v_ay_1 = v_ay_1.to_Matrix(vector_only=True)
v_az_1 = (2 * qw_e_dot_az_1[1]).mul(qw_e_dot_az_1[0].conjugate())
v_az_1 = v_az_1.to_Matrix(vector_only=True)



In [ ]:
test = v_theta_1.subs(test_subs)
display(Math(sympy.latex(test)))

<IPython.core.display.Math object>

##Joint 2



In [ ]:
r_dot_theta_2 = dualq_diff_alt(Q_f, theta_2)
r_dot_ax_2 = dualq_diff_alt(Q_f, ax_2)
r_dot_ay_2 = dualq_diff_alt(Q_f, ay_2)
r_dot_az_2 = dualq_diff_alt(Q_f, az_2)

qw_e_dot_theta_2 = dualq_mul(Q_c, r_dot_theta_2, Q_t, Q_e)
qw_e_dot_ax_2 = dualq_mul(Q_c, r_dot_ax_2, Q_t, Q_e)
qw_e_dot_ay_2 = dualq_mul(Q_c, r_dot_ay_2, Q_t, Q_e)
qw_e_dot_az_2 = dualq_mul(Q_c, r_dot_az_2, Q_t, Q_e)

### Joint 2 Angular Velocity

In [ ]:
w_theta_2_1 = ((o_l_dot_r_1.conjugate()).mul(qw_e_dot_theta_2[0])).scalar_part()
w_theta_2_2 = ((o_l_dot_r_2.conjugate()).mul(qw_e_dot_theta_2[0])).scalar_part()
w_theta_2_3 = ((o_l_dot_r_3.conjugate()).mul(qw_e_dot_theta_2[0])).scalar_part()
w_theta_2 = sympy.Matrix([w_theta_2_1, w_theta_2_2, w_theta_2_3]) * 2

w_ax_2_1 = ((o_l_dot_r_1.conjugate()).mul(qw_e_dot_ax_2[0])).scalar_part()
w_ax_2_2 = ((o_l_dot_r_2.conjugate()).mul(qw_e_dot_ax_2[0])).scalar_part()
w_ax_2_3 = ((o_l_dot_r_3.conjugate()).mul(qw_e_dot_ax_2[0])).scalar_part()
w_ax_2 = sympy.Matrix([w_ax_2_1, w_ax_2_2, w_ax_2_3]) * 2

w_ay_2_1 = ((o_l_dot_r_1.conjugate()).mul(qw_e_dot_ay_2[0])).scalar_part()
w_ay_2_2 = ((o_l_dot_r_2.conjugate()).mul(qw_e_dot_ay_2[0])).scalar_part()
w_ay_2_3 = ((o_l_dot_r_3.conjugate()).mul(qw_e_dot_ay_2[0])).scalar_part()
w_ay_2 = sympy.Matrix([w_ay_2_1, w_ay_2_2, w_ay_2_3]) * 2

w_az_2_1 = ((o_l_dot_r_1.conjugate()).mul(qw_e_dot_az_2[0])).scalar_part()
w_az_2_2 = ((o_l_dot_r_1.conjugate()).mul(qw_e_dot_az_2[0])).scalar_part()
w_az_2_3 = ((o_l_dot_r_1.conjugate()).mul(qw_e_dot_az_2[0])).scalar_part()
w_az_2 = sympy.Matrix([w_az_2_1, w_az_2_2, w_az_2_3]) * 2

In [ ]:
test = w_theta_2.subs(test_subs)
display(Math(sympy.latex(test)))

<IPython.core.display.Math object>

### Joint 2 Linear Velocity

In [ ]:
'''
v_theta_2_a = 2 * qw_e_dot_theta_2[1]
v_theta_2_b = q_ew_t.mul(qw_e_dot_theta_2[0])
v_theta_2 = (q_subtract(v_theta_2_a, v_theta_2_b)).mul(q_ew_r.conjugate())
v_theta_2 = v_theta_2.to_Matrix(vector_only=True)

v_ax_2_a = 2 * qw_e_dot_ax_2[1]
v_ax_2_b = q_ew_t.mul(qw_e_dot_ax_2[0])
v_ax_2 = (q_subtract(v_ax_2_a, v_ax_2_b)).mul(q_ew_r.conjugate())
v_ax_2 = v_ax_2.to_Matrix(vector_only=True)

v_ay_2_a = 2 * qw_e_dot_ay_2[1]
v_ay_2_b = q_ew_t.mul(qw_e_dot_ay_2[0])
v_ay_2 = (q_subtract(v_ay_2_a, v_ay_2_b)).mul(q_ew_r.conjugate())
v_ay_2 = v_ay_2.to_Matrix(vector_only=True)

v_az_2_a = 2 * qw_e_dot_az_2[1]
v_az_2_b = q_ew_t.mul(qw_e_dot_az_2[0])
v_az_2 = (q_subtract(v_az_2_a, v_az_2_b)).mul(q_ew_r.conjugate())
v_az_2 = v_az_2.to_Matrix(vector_only=True)
'''
v_theta_2 = (2 * qw_e_dot_theta_2[1]).mul(qw_e_dot_theta_2[0].conjugate())
v_theta_2 = v_theta_2.to_Matrix(vector_only=True)
v_ax_2 = (2 * qw_e_dot_ax_2[1]).mul(qw_e_dot_ax_2[0].conjugate())
v_ax_2 = v_ax_2.to_Matrix(vector_only=True)
v_ay_2 = (2 * qw_e_dot_ay_2[1]).mul(qw_e_dot_ay_2[0].conjugate())
v_ay_2 = v_ay_2.to_Matrix(vector_only=True)
v_az_2 = (2 * qw_e_dot_az_2[1]).mul(qw_e_dot_az_2[0].conjugate())
v_az_2 = v_az_2.to_Matrix(vector_only=True)




In [ ]:
test = v_theta_2.subs(test_subs)
display(Math(sympy.latex(test)))

<IPython.core.display.Math object>

##Joint 3

In [ ]:
r_dot_theta_3 = dualq_diff_alt(Q_t, theta_3)
r_dot_ax_3 = dualq_diff_alt(Q_t, ax_3)
r_dot_ay_3 = dualq_diff_alt(Q_t, ay_3)
r_dot_az_3 = dualq_diff_alt(Q_t, az_3)

q_we_dot_theta_3 = dualq_mul(Q_c, Q_f, r_dot_theta_3, Q_e)
q_we_dot_ax_3 = dualq_mul(Q_c, Q_f, r_dot_ax_3, Q_e)
q_we_dot_ay_3 = dualq_mul(Q_c, Q_f, r_dot_ay_3, Q_e)
q_we_dot_az_3 = dualq_mul(Q_c, Q_f, r_dot_az_3, Q_e)

###Joint 3 Angular Velocity

In [ ]:
w_theta_3_1 = ((o_l_dot_r_1.conjugate()).mul(q_we_dot_theta_3[0])).scalar_part()
w_theta_3_2 = ((o_l_dot_r_2.conjugate()).mul(q_we_dot_theta_3[0])).scalar_part()
w_theta_3_3 = ((o_l_dot_r_3.conjugate()).mul(q_we_dot_theta_3[0])).scalar_part()
w_theta_3 = sympy.Matrix([w_theta_3_1, w_theta_3_2, w_theta_3_3]) * 2

w_ax_3_1 = ((o_l_dot_r_1.conjugate()).mul(q_we_dot_ax_3[0])).scalar_part()
w_ax_3_2 = ((o_l_dot_r_2.conjugate()).mul(q_we_dot_ax_3[0])).scalar_part()
w_ax_3_3 = ((o_l_dot_r_3.conjugate()).mul(q_we_dot_ax_3[0])).scalar_part()
w_ax_3 = sympy.Matrix([w_ax_3_1, w_ax_3_2, w_ax_3_3]) * 2

w_ay_3_1 = ((o_l_dot_r_1.conjugate()).mul(q_we_dot_ay_3[0])).scalar_part()
w_ay_3_2 = ((o_l_dot_r_2.conjugate()).mul(q_we_dot_ay_3[0])).scalar_part()
w_ay_3_3 = ((o_l_dot_r_3.conjugate()).mul(q_we_dot_ay_3[0])).scalar_part()
w_ay_3 = sympy.Matrix([w_ay_3_1, w_ay_3_2, w_ay_3_3]) * 2

w_az_3_1 = ((o_l_dot_r_1.conjugate()).mul(q_we_dot_az_3[0])).scalar_part()
w_az_3_2 = ((o_l_dot_r_2.conjugate()).mul(q_we_dot_az_3[0])).scalar_part()
w_az_3_3 = ((o_l_dot_r_3.conjugate()).mul(q_we_dot_az_3[0])).scalar_part()
w_az_3 = sympy.Matrix([w_az_3_1, w_az_3_2, w_az_3_3]) * 2

In [ ]:
test = w_theta_3.subs(test_subs)
display(Math(sympy.latex(test)))

<IPython.core.display.Math object>

###Joint 3 Linear Velocity

In [ ]:
'''
v_theta_3_a = 2 * q_we_dot_theta_3[1]
v_theta_3_b = q_ew_t.mul(q_we_dot_theta_3[0])
v_theta_3 = (q_subtract(v_theta_3_a, v_theta_3_b)).mul(q_ew_r.conjugate())
v_theta_3 = v_theta_3.to_Matrix(vector_only=True)

v_ax_3_a = 2 * q_we_dot_ax_3[1]
v_ax_3_b = q_ew_t.mul(q_we_dot_ax_3[0])
v_ax_3 = (q_subtract(v_ax_3_a, v_ax_3_b)).mul(q_ew_r.conjugate())
v_ax_3 = v_ax_3.to_Matrix(vector_only=True)

v_ay_3_a = 2 * q_we_dot_ay_3[1]
v_ay_3_b = q_ew_t.mul(q_we_dot_ay_3[0])
v_ay_3 = (q_subtract(v_ay_3_a, v_ay_3_b)).mul(q_ew_r.conjugate())
v_ay_3 = v_ay_3.to_Matrix(vector_only=True)

v_az_3_a = 2 * q_we_dot_az_3[1]
v_az_3_b = q_ew_t.mul(q_we_dot_az_3[0])
v_az_3 = (q_subtract(v_az_3_a, v_az_3_b)).mul(q_ew_r.conjugate())
v_az_3 = v_az_3.to_Matrix(vector_only=True)
'''
v_theta_3 = (2 * q_we_dot_theta_3[1]).mul(q_we_dot_theta_3[0].conjugate())
v_theta_3 = v_theta_3.to_Matrix(vector_only=True)
v_ax_3 = (2 * q_we_dot_ax_3[1]).mul(q_we_dot_ax_3[0].conjugate())
v_ax_3 = v_ax_3.to_Matrix(vector_only=True)
v_ay_3 = (2 * q_we_dot_ay_3[1]).mul(q_we_dot_ay_3[0].conjugate())
v_ay_3 = v_ay_3.to_Matrix(vector_only=True)
v_az_3 = (2 * q_we_dot_az_3[1]).mul(q_we_dot_az_3[0].conjugate())
v_az_3 = v_az_3.to_Matrix(vector_only=True)


In [ ]:
test = v_theta_3.subs(test_subs)
display(Math(sympy.latex(test)))

<IPython.core.display.Math object>

## Jacobian Definition

In [ ]:
joint_1_v = sympy.Matrix.hstack(v_theta_1, v_ax_1, v_ay_1, v_az_1)
joint_2_v = sympy.Matrix.hstack(v_theta_2, v_ax_2, v_ay_2, v_az_2)
joint_3_v = sympy.Matrix.hstack(v_theta_2, v_ax_3, v_ay_3, v_az_3)

joint_1_w = sympy.Matrix.hstack(w_theta_1, w_ax_1, w_ay_1, w_az_1)
joint_2_w = sympy.Matrix.hstack(w_theta_2, w_ax_2, w_ay_2, w_az_2)
joint_3_w = sympy.Matrix.hstack(w_theta_3, w_ax_3, w_ay_3, w_az_3)


In [ ]:
linear_velocities = sympy.Matrix.hstack(joint_1_v, joint_2_v, joint_3_v)
angular_velocities = sympy.Matrix.hstack(joint_1_w, joint_2_w, joint_3_w)
J = sympy.Matrix.vstack(linear_velocities, angular_velocities)
sympy.shape(J)

(6, 12)

In [ ]:
variables = [theta_1, ax_1, ay_1, az_1,theta_2, ax_2, ay_2, az_2, theta_3, ax_3, ay_3, az_3]
J_Lambda = sympy.lambdify(variables, J, modules=["numpy"], cse=True, docstring_limit=0, dummify=False)

In [ ]:
??J_Lambda

#Solver

In [ ]:
variables = [theta_1, ax_1, ay_1, az_1,theta_2, ax_2, ay_2, az_2, theta_3, ax_3, ay_3, az_3]
foot_position_FK_lambda = sympy.lambdify(variables, foot_position_FK, modules=["numpy"], cse=True, docstring_limit=0)
foot_orientation_FK_lambda = sympy.lambdify(variables, foot_orientation_FK, modules=["numpy"], cse=True, docstring_limit=0)

In [ ]:
def wrap(current_angles):

    if current_angles[0] > np.pi/2 :
      current_angles[0] = current_angles[0] - np.pi
    if current_angles[4] > np.pi/2:
      current_angles[4] = current_angles[4] - np.pi
    if current_angles[8] > np.pi:
      current_angles[8] = current_angles[8] - np.pi

    if current_angles[0] < -np.pi/2:
      current_angles[0] = current_angles[0] + np.pi
    if current_angles[4] < -np.pi/2:
      current_angles[4] = current_angles[4] + np.pi
    if current_angles[8] <  0:
      current_angles[8] = current_angles[8] + np.pi


    return current_angles

def wrap_to_2_pi(current_angles):

    if current_angles[0] > 2 * np.pi:
      current_angles[0] = current_angles[0] - 2 * np.pi
    if current_angles[4] > 2 * np.pi:
      current_angles[4] = current_angles[4] - 2 * np.pi
    if current_angles[8] > 2 * np.pi:
      current_angles[8] = current_angles[8] - 2 * np.pi

    if current_angles[0] < -2 * np.pi:
      current_angles[0] = current_angles[0] + 2 * np.pi
    if current_angles[4] < -2 * np.pi:
      current_angles[4] = current_angles[4] + 2 * np.pi
    if current_angles[8] < -2 * np.pi:
      current_angles[8] = current_angles[8] + 2 * np.pi


    return current_angles

def wrap_to_pi(angle_changes):
  """Wraps an angle to the range (-pi, pi]."""
  angle_changes[0] = (angle_changes[0] + np.pi) % (2 * np.pi) - np.pi
  angle_changes[4] = (angle_changes[4] + np.pi) % (2 * np.pi) - np.pi
  angle_changes[8] = (angle_changes[8] + np.pi) % (2 * np.pi) - np.pi
  return angle_changes

def normalize_axis_vectors(current_angles):
  axis_vector_1 = current_angles[1:4]
  current_angles[1:4] = axis_vector_1 / np.linalg.norm(axis_vector_1)
  axis_vector_2 = current_angles[5:8]
  current_angles[5:8] = axis_vector_2 / np.linalg.norm(axis_vector_2)
  axis_vector_3 = current_angles[9:12]
  current_angles[9:12] = axis_vector_3 / np.linalg.norm(axis_vector_3)
  return current_angles



In [ ]:
from scipy.spatial.transform import Rotation as R

# The foot orientation quaternion from your output
foot_orientation_quat = np.array([0.86928226, -0.15165949, 0.17372518, 0.45357814])

# Create a Rotation object
r = R.from_quat(foot_orientation_quat)

# Convert to Euler angles (in radians) - Assuming XYZ convention
euler_angles = r.as_euler('xyz')

# Convert to degrees for easier understanding
euler_angles_degrees = np.rad2deg(euler_angles)

print("Foot Orientation (Euler angles in degrees):", euler_angles_degrees)

Foot Orientation (Euler angles in degrees): [126.41011865 -25.67716478  -6.66169564]


In [ ]:

angle_in_radians = 2.699289958818831

angle_in_degrees = degrees(angle_in_radians)

print(angle_in_degrees)

154.6579223223608


In [ ]:
def solve_ik(current_angles, target_ep, max_error, max_iterations,delta_t, secondary_weight, gamma, penalty):
  # Set convergence criteria

  DEBUG = True
  ERROR_DEBUG = False
  SECONDARY_DEBUG = False

  STARTING_FOOTPOSE = None
  DAMP = True

  THETA_MAX = np.array([np.pi/2, 0., 0., 1., np.pi/2, 0., 1., 0., np.pi, 0., 1., 0.], dtype=np.float64)
  THETA_MIN = np.array([-np.pi/2, 0., 0., 0.9, -np.pi/2, 0., 0.9, 0., 0, 0., 0.9, 0.], dtype=np.float64)
  LM_START = np.zeros(12)

  converged = False



  #Current Angles are in the zero position

  analytics_df = pd.DataFrame()

  for i in range(max_iterations):

    analytics = {'delta_t':delta_t*i}

    if DEBUG or ERROR_DEBUG or SECONDARY_DEBUG:
      print("______________________________________________________________________________________________________________________________________________________________")
      print(f"Iteration: {i} Started")



    # 1. Substitute current joint angles into the desired pose
    current_foot_position = foot_position_FK_lambda(*current_angles).astype(np.float64)
    current_foot_orientation = foot_orientation_FK_lambda(*current_angles).astype(np.float64)
    current_foot_pose = np.concatenate((current_foot_position, current_foot_orientation)).flatten()

    if i == 0:
      STARTING_FOOTPOSE = current_foot_pose
      if DEBUG: print(f"STARTING_FOOTPOSE: {STARTING_FOOTPOSE}")



    if DEBUG or ERROR_DEBUG:
      print(f"current_foot_pose: {current_foot_pose}")
      print(f"target_foot_pose: {target_ep}")

    # 2. Calculate error between current and desired pose
    t_error = current_foot_pose[0:3] - target_ep[0:3]
    o_error = current_foot_pose[3:] - target_ep[3:]

    error = np.concatenate((o_error, t_error))

    if DEBUG or ERROR_DEBUG:
      print(f"error = {error}")

    error_norm = np.linalg.norm(t_error)
    analytics['error_norm'] = error_norm


    # 3. Check for convergence
    if error_norm < max_error :
          print("Converged!")
          converged = True
          break
    if DEBUG or ERROR_DEBUG: print(f"error.norm() = {error_norm}")



    # 4. Substitute current joint angles into the Jacobian
    J_num = J_Lambda(*current_angles).astype(np.float64)
    if DEBUG or ERROR_DEBUG: print(f"J_num = {J_num}")

    cond_num = np.linalg.cond(J_num)
    if DEBUG or ERROR_DEBUG: print(f"Condition Number = {cond_num}")

    analytics['cond_num'] = cond_num

    rank = np.linalg.matrix_rank(J_num)
    if DEBUG or ERROR_DEBUG: print(f"Rank = {rank}")

    analytics['rank'] = rank

    # 5. Calculate joint velocities

    if (rank < 6 or cond_num > 400) and DAMP:
      if DEBUG: print("Using damped Jacobian")
      J_damped = J_num.T @ np.linalg.inv(J_num @ J_num.T + gamma**2 * np.eye(J_num.shape[0]))
      if DEBUG: print(f"J_damped = {J_damped}")
      primary_velocity = J_damped @ error
    else:
      if DEBUG: print("Using Undamped Jacobian")
      J_pinv = np.linalg.pinv(J_num)
      if DEBUG: print(f"J_pinv = {J_pinv}")
      primary_velocity = J_pinv @ error

    upper_violation = np.maximum(current_angles - THETA_MAX, 0)
    lower_violation = np.maximum(THETA_MIN - current_angles, 0)
    current_violations = np.where(upper_violation > 0, upper_violation, 0) + \
                  np.where(lower_violation > 0, lower_violation, 0)

    lm = np.maximum(0, LM_START + penalty * current_violations)
    if SECONDARY_DEBUG: print(f"LM = {lm}")

    epsilon = 1e-6
    gradient = np.zeros_like(current_angles)

    for j in range(len(current_angles)):
        perturbed_angles = current_angles.copy()
        perturbed_angles[j] += epsilon

        upper_violation_p = np.maximum(perturbed_angles - THETA_MAX, 0)
        lower_violation_p = np.maximum(THETA_MIN - perturbed_angles, 0)
        perturbed_violations = np.where(upper_violation_p > 0, upper_violation_p, 0) + \
                    np.where(lower_violation_p > 0, lower_violation_p, 0)

        perturbed_L = error_norm**2 + lm * perturbed_violations.T + (penalty/2) * np.linalg.norm(perturbed_violations)**2
        original_L = error_norm**2+ lm * current_violations.T + (penalty/2) * np.linalg.norm(current_violations)**2
        gradient[j] = (perturbed_L[j]-original_L[j]) / epsilon

    secondary_velocity = gradient
    if SECONDARY_DEBUG: print(f"primary_velocity = {primary_velocity}")
    if SECONDARY_DEBUG: print(f"secondary_velocity = {secondary_velocity}")

    q_dot = primary_velocity + secondary_weight * secondary_velocity

    if DEBUG or ERROR_DEBUG: print(f"q_dot = {q_dot}")

    delta_theta = q_dot * delta_t
    if DEBUG or ERROR_DEBUG: print(f"delta_theta = {delta_theta}")

    # 6. Update joint angles (integrate velocities)
    if DEBUG: print(f"previous_angles = {current_angles}")
    current_angles = current_angles + delta_theta

    current_angles = wrap_to_2_pi(current_angles)

    #current_angles = normalize_axis_vectors(current_angles)

    if DEBUG: print(f"new_angles = {current_angles}")



    analytics['Tx'] = current_foot_pose[0]
    analytics['Ty'] = current_foot_pose[1]
    analytics['Tz'] = current_foot_pose[2]
    analytics['Ox'] = current_foot_pose[3]
    analytics['Oy'] = current_foot_pose[4]
    analytics['Oz'] = current_foot_pose[5]


    analytics['delta_theta_1'] = delta_theta[0]
    analytics['delta_ax_1'] = delta_theta[1]
    analytics['delta_ay_1'] = delta_theta[2]
    analytics['delta_az_1'] = delta_theta[3]
    analytics['delta_theta_2'] = delta_theta[4]
    analytics['delta_ax_2'] = delta_theta[5]
    analytics['delta_ay_2'] = delta_theta[6]
    analytics['delta_az_2'] = delta_theta[7]
    analytics['delta_theta_3'] = delta_theta[8]
    analytics['delta_ax_3'] = delta_theta[9]
    analytics['delta_ay_3'] = delta_theta[10]
    analytics['delta_az_3'] = delta_theta[11]

    analytics['theta_1'] = current_angles[0]
    analytics['ax_1'] = current_angles[1]
    analytics['ay_1'] = current_angles[2]
    analytics['az_1'] = current_angles[3]
    analytics['theta_2'] = current_angles[4]
    analytics['ax_2'] = current_angles[5]
    analytics['ay_2'] = current_angles[6]
    analytics['az_2'] = current_angles[7]
    analytics['theta_3'] = current_angles[8]
    analytics['ax_3'] = current_angles[9]
    analytics['ay_3'] = current_angles[10]
    analytics['az_3'] = current_angles[11]


    analytics_df = pd.concat([analytics_df, pd.DataFrame([analytics])], ignore_index=True)
    if DEBUG or ERROR_DEBUG or SECONDARY_DEBUG:
      print(f"current_angles = {current_angles}")
      print(f"Iteration: {i} ended")
      print("______________________________________________________________________________________________________________________________________________________________")


  # If we didn't converge, let the user know
  if i == max_iterations - 1:
      print("Failed to converge.")
  print(f"current_angles = {repr(current_angles)}")
  print(f"Iteration # = {i}")
  print(f"Distance = {np.linalg.norm(target_ef - STARTING_FOOTPOSE)}")
  # Print the final joint angles (whether converged or not)
  print("Final joint angles:")
  print(f"cond: {analytics_df['cond_num'].mean()}")
  print(f"min error {analytics_df['error_norm'].min()}")
  print(f"Joint 1: {current_angles[0]}")
  print(f"Joint 2: {current_angles[4]}")
  print(f"Joint 3: {current_angles[8]}")

  return analytics_df, analytics_df['error_norm'].min(), current_angles, converged

In [ ]:
current_angles = np.array([1e-8, 0, 0, 1, 1e-8, 0, 1, 0, 1e-8, 0, 1, 0], dtype=np.float64)
test_angles = np.array([0, 0, 0, 1, 0, 0, 1, 0, pi/2, 0, 1, 0], dtype=np.float64)
zero = 1e-8
target_ef = np.array([4.26325641e-14, 2.30000000e+02, 0.00000000e+00, 0.        , 0.        , 0.70710678], dtype=np.float64)
MAX_ERROR = 1.5
MAX_ITERATIONS = 200
DELTA_T = 0.01
SECONDARY_WEIGHT = 0
GAMMA = 0.1
PENALTY = 0.9

best_params = None
best_score = np.inf

delta_t_params = [0.1, 0.01, 0.001, 0.0001]
secondary_weight_params = [0, 0.1, 0.25, 0.5, 0.75, 1., 1.25, 1.5]
gamma_params = [0.1, 0.01, 0.001, 0.0001]
penalty_params = [0.1, 0.01, 0.001, 0.0001]

for delta_t in delta_t_params:
    for secondary_weight in secondary_weight_params:
        for gamma in gamma_params:
            for penalty in penalty_params:
                current_angles = np.array([1e-8, 0, 0, 1, 1e-8, 0, 1, 0, 1e-8, 0, 1, 0], dtype=np.float64)
                print("______________________________________________________________________________________________________________________________________________________________")
                analytics, error_norm, current_angles, converged = solve_ik(current_angles, target_ef, MAX_ERROR,
                                                                MAX_ITERATIONS,delta_t, secondary_weight, gamma, penalty)
                angle_norm = np.linalg.norm(current_angles - test_angles)
                print(f"angle_error = {angle_norm}")
                print("______________________________________________________________________________________________________________________________________________________________")
                if angle_norm < best_score:
                  best_score = angle_norm
                  best_params = (delta_t, secondary_weight, gamma, penalty, analytics)


analytics_df = best_params[4]

Streaming output truncated to the last 5000 lines.
Joint 1: -1.673425428571973
Joint 2: -3.4936688895054875
Joint 3: 3.7340732119561357
angle_error = 66.19976859758076
______________________________________________________________________________________________________________________________________________________________
______________________________________________________________________________________________________________________________________________________________
Failed to converge.
current_angles = array([-1.67646249e+00,  4.56459557e-02,  3.33917596e+00,  5.99864799e-02,
       -3.49835036e+00, -7.28637010e+00, -7.76633375e-01,  6.50621747e+01,
        3.73714037e+00,  3.76561608e+00, -1.84724813e+00,  4.47700158e+00])
Iteration # = 199
Distance = 325.269886312876
Final joint angles:
cond: 4.6544889254040227e+21
min error 113.78229077699847
Joint 1: -1.6764624868222953
Joint 2: -3.4983503618380873
Joint 3: 3.7371403734182995
angle_error = 66.05635665718569
_________

In [ ]:
print(f"Best Score: {best_score}")
print(f"Best Params: {best_params[:4]}")

Best Score: 4.389905334078707
Best Params: (0.001, 1.5, 0.1, 0.001)


In [ ]:
test_angles = np.array([pi/8, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0], dtype=np.float64)
print(f'Foot Position: {repr(foot_position_FK_lambda(*test_angles).flatten())}')
print(f'Foot Orientation: {repr(foot_orientation_FK_lambda(*test_angles).flatten())}')

Foot Position: array([212.49229248,  88.01718944,   0.        ])
Foot Orientation: array([0.        , 0.        , 0.19509032])


In [ ]:
zero = 1e-8

current_angles = np.array([zero, 0, 0, 1, zero, 0, 1, 0, zero, 0, 1, 0], dtype=np.float64)
target_ef = np.array([212.49229248,  88.01718944,   0.        , 0.        , 0.        , 0.19509032], dtype=np.float64)
max_error = 1
MAX_ITERATIONS = 500
delta_t = 0.001
secondary_weight = 1.5
gamma = 0.001
penalty = 0.001

analytics_df, error_norm, current_angles, converged = solve_ik(current_angles, target_ef, max_error,
                                                               MAX_ITERATIONS,delta_t, secondary_weight, gamma, penalty)

Streaming output truncated to the last 5000 lines.
   8.47328683e+00 -9.62285513e+00]]
q_dot = [-8.46486823e-02  7.13735496e+00 -1.15285776e+01 -1.14910534e+01
  6.86272370e-02  6.56824755e+00 -4.62119209e+01 -1.49291895e+03
 -5.94289832e-02 -6.24879845e+00  4.92716916e+00  1.11849105e+02]
delta_theta = [-8.46486823e-05  7.13735496e-03 -1.15285776e-02 -1.14910534e-02
  6.86272370e-05  6.56824755e-03 -4.62119209e-02 -1.49291895e+00
 -5.94289832e-05 -6.24879845e-03  4.92716916e-03  1.11849105e-01]
previous_angles = [-1.93207918e-01 -5.48460806e+01 -2.57504856e+01  1.88191732e+00
  2.84046805e+00 -5.75349628e+00 -2.21286018e+02 -1.22632733e+03
 -3.05925823e+00 -7.57094074e+01  5.35371687e+02 -3.96748835e+01]
new_angles = [-1.93292567e-01 -5.48389433e+01 -2.57620142e+01  1.87042627e+00
  2.84053668e+00 -5.74692804e+00 -2.21332230e+02 -1.22782025e+03
 -3.05931766e+00 -7.57156562e+01  5.35376614e+02 -3.95630343e+01]
current_angles = [-1.93292567e-01 -5.48389433e+01 -2.57620142e+01  1.8704262

In [ ]:
alt.renderers.set_embed_options(theme='dark')


joint_angles = ['theta_1', 'theta_2', 'theta_3']
end_effector_position = ['Tx', 'Ty', 'Tz']
joint_angles_df = analytics_df[joint_angles + ['delta_t']]
end_effector_position_df = analytics_df[end_effector_position + ['delta_t']]

end_effector_orientation = ['Ox', 'Oy', 'Oz']
end_effector_orientation_df = analytics_df[end_effector_orientation + ['delta_t']]

graph_height = 375
graph_width = 650

analytics_df['error_norm_ma'] = analytics_df['error_norm'].rolling(window=10).mean()
analytics_df['cond_num_ma'] = analytics_df['cond_num'].rolling(window=10).mean()

error_base = alt.Chart(analytics_df).encode(
    x='delta_t:Q',
    tooltip=['delta_t']
)

error_line = error_base.mark_line().encode(
    y='error_norm:Q',
    tooltip=['error_norm']
)

ma_line = error_base.mark_line(color='red', strokeDash=[5,5]).encode(
    y='error_norm_ma:Q',
    tooltip=['error_norm_ma']
)

error_chart = alt.layer(error_line, ma_line).properties(
    title='Error Over Time',
    width=graph_width,
    height=graph_height
).interactive()

cond_num_base = alt.Chart(analytics_df).encode(
    x='delta_t:Q',
    tooltip=['delta_t']
)

cond_num_line = error_base.mark_line().encode(
    y=alt.Y("cond_num:Q", axis=alt.Axis(tickCount=8, format=".1e")),
    tooltip=['cond_num']
)

cond_num_ma_line = error_base.mark_line(color='red', strokeDash=[5,5]).encode(
    y='cond_num_ma:Q',
    tooltip=['cond_num_ma']
)

cond_num_chart = alt.layer(cond_num_line, cond_num_ma_line).properties(
    title='Condition Number Over Time',
    width=graph_width,
    height=graph_height
).interactive()

rank_chart = alt.Chart(analytics_df).mark_line().encode(
    x='delta_t:Q',
    y='rank:Q',
    tooltip=['delta_t', 'cond_num']
).properties(
    title='Matrix Rank Over Time',
    width=graph_width,
    height=graph_height
).interactive()

joint_angle_chart = alt.Chart(joint_angles_df).transform_fold(
    joint_angles,
    as_=['joint', 'angle']
).mark_line().encode(
    x='delta_t:Q',
    y=alt.Y('angle:Q', title='Angle (radians)'),
    color=alt.Color('joint:N', scale=alt.Scale(scheme='dark2')),
    tooltip=['delta_t:Q', 'angle:Q']
).properties(
    title='Joint Angles Over Time',
    width=graph_width,
    height=graph_height
).interactive()

orientation_chart = alt.Chart(end_effector_orientation_df).transform_fold(
    end_effector_orientation,
    as_=['axis', 'value']
).mark_line().encode(
    x='delta_t:Q',
    y=alt.Y('value:Q', title='Orientation'),
    color=alt.Color('axis:N', scale=alt.Scale(scheme='dark2')),
    tooltip=['delta_t:Q', 'value:Q']
).properties(
    title='End-Effector Orientation Over Time',
    width=graph_width,
    height=graph_height
).interactive()

position_chart = alt.Chart(end_effector_position_df).transform_fold(
    end_effector_position,
    as_=['axis', 'value']
).mark_line().encode(
    x='delta_t:Q',
    y=alt.Y('value:Q', title='Position'),
    color= alt.Color('axis:N', scale=alt.Scale(scheme='dark2')),
    tooltip=['delta_t:Q', 'value:Q']
).properties(
    title='End-Effector Position Over Time',
    width=graph_width,
    height=graph_height
).interactive()

row_1 = alt.hconcat(error_chart, cond_num_chart, rank_chart)
row_2 = alt.hconcat(orientation_chart, position_chart, joint_angle_chart)

dashboard = alt.vconcat(row_1, row_2).resolve_scale(
    color='independent'
).configure_legend(
    titleFontSize=18,
    labelFontSize=15
).configure_axis(
    titleFontSize=18,
    labelFontSize=15
)

dashboard.title = alt.TitleParams(f'Target: Tx: {target_ef[0]:.2f}  Ty: {target_ef[1]:.2f}  Tz: {target_ef[2]:.2f} Ox: {target_ef[3]:.2f} Oy: {target_ef[4]:.2f} Oz: {target_ef[5]:.2f}', anchor='middle')

dashboard.display()


alt.VConcatChart(...)